<a href="https://colab.research.google.com/github/ezeguins/Bounding-Box-Detecction-VGG16/blob/main/dataset_generator_ts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GENERATE .NPZ DATASET FILE FROM TWO DIFFERENT CVS ANNOTATIONS FILES
First dataset genertion and second dataset generation attached to the first one.
Record file to current gdrive 



In [ ]:
import os
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [ ]:
BASE_PATH = "/content/gdrive/MyDrive/VGG16/dataset"
IMAGES_PATH = os.path.sep.join([BASE_PATH, "images"])
ANNOTS_PATH = os.path.sep.join([BASE_PATH, "etiquetas_export-data-60_solobloques.csv"])
IMAGES_PATH2 = "/content/gdrive/MyDrive/segundoset/finales"
ANNOTS_PATH2 = "/content/gdrive/MyDrive/segundoset/etiquetas-segundos-export.csv"

# define the path to the base output directory
BASE_OUTPUT = "/content/gdrive/MyDrive/VGG16/output"
# define the path to the output serialized model, model training plot,
# and testing image filenames

In [ ]:
# load the contents of the CSV annotations file
print("[INFO] loading dataset...")
rows = open(ANNOTS_PATH).read().strip().split("\n")
rows = [w.replace(';', ',') for w in rows]
# initialize the list of data (images), our target output predictions (bounding box coordinates), along with the filenames of the
# individual images
data = []
targets = []
filenames = []

In [ ]:
# FIRST DATASET GENERATION

# loop over the rows
for idx, row in enumerate(rows):
	# break the row into the filename and bounding box coordinates
  row = row.split(",")
  #print(row)
  (filename,c,startX,c,  endX,c, startY,c, endY) = row
  if startX != "0" or startX != "" :
    print(idx, filename, "startX",startX, "endX", endX, "startY", startY, "endY", endY)
    # derive the path to the input image, load the image (in OpenCV format), and grab its dimensions
    imagePath = os.path.sep.join([IMAGES_PATH, filename])
     # image = cv2.imread(imagePath)
    #(h, w) = image.shape[:2] - no hace falta porque nuestras anotaciones ya están escaladas
    # load the image and preprocess it
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
	  # update our list of data, targets, and filenames
    data.append(image)
    targets.append((startX, startY, endX, endY))
    filenames.append(filename)

In [ ]:
# INITIALIZE SECOND DATASET:

print("[INFO] loading dataset...")
rows = open(ANNOTS_PATH2).read().strip().split("\n")
rows = [w.replace(';', ',') for w in rows]



In [ ]:
# SECOND DATASET GENERATION. APPEND TO FIRST DATASET INCLUDED

from pathlib import Path

# loop over the rows
for idx, row in enumerate(rows):
# break the row into the filename and bounding box coordinates
  row = row.split(",")
  #print(row)
  (filename,c,startX,c,  endX,c, startY,c, endY) = row
  
  # derive the path to the input image, load the image (in OpenCV format), and grab its dimensions
  imagePath = os.path.sep.join([IMAGES_PATH2, filename])
  imagePath = imagePath.replace("\ufeff","")
  my_file = Path(imagePath)
  if my_file.is_file():
    # image = cv2.imread(imagePath)
    #(h, w) = image.shape[:2] - no hace falta porque nuestras anotaciones ya están escaladas
    # load the image and preprocess it
    if startX != "0":
      image = load_img(imagePath, target_size=(224, 224))
      image = img_to_array(image)
      print(idx, filename, "startX",startX, "endX", endX, "startY", startY, "endY", endY)
	    # update our list of data, targets, and filenames
      data.append(image)
      targets.append((startX, startY, endX, endY))
      filenames.append(filename)
  else:
    print(filename, "no existe")

In [ ]:
# ANNOTATION CONVERSION. FROM STRING TO FLOAT32

res = []
for tup in targets:
  temp = []
  for ele in tup:
    if ele=="":
      temp.append(0)
    else:
      temp.append(float(ele))
  res.append((temp[0],temp[1],temp[2],temp[3]))

In [ ]:
# SAVE TO DGRIVE THE DATASET FILE .NPZ

inicio = 0
fin = len(res)
print(fin)
# convert the data and targets to NumPy arrays, scaling the input
# pixel intensities from the range [0, 255] to [0, 1]
data1 = np.array(data[inicio: fin], dtype="float32") / 255.0
targets1 = np.array(res[inicio: fin], dtype="float32")
filenames1 = filenames[inicio: fin]
outfile = "/content/gdrive/MyDrive/VGG16/dataset/datos.npz"
np.savez(outfile, data1, targets1,filenames1)